# A simple tool for quantum creativity

One of the simplest things we can do with a quantum computer is to create interference effects. Here we will make some simple tools to help us do this.

We will be making these tools for MicroQiskit. For a guide that uses Qiskit, see [here](https://github.com/quantumjim/blog/blob/master/Quantum_Procedural_Generation/3_FewQubit.ipynb).

In [48]:
from microqiskit import QuantumCircuit, simulate
from math import pi, ceil, log, sqrt

Lots of different types of information can be nicely expressed as a list of numbers. Such as in a 2D platform game, where the terrain could be described by a list of heights.

For example, the following list could describe terrain that starts off at a height 0.5, slowly rises up to 1.0 and then quickly goes back down to 0.5.

In [67]:
height = [0.5,0.63,0.77,1,0.75,0.5]

In this list, two neighbouring entries correspond to two neighbouring points. So we would expect that neighbouring entries will probably be somewhat similar. And if we change the list in some way, it would be good to maintain the similarity between neighbouring points.

Another example could be the volume of a piece of music over time, perhaps expressed as a list of the volumes for each beat. Or it could be a list of brightnesses for a line of pixels. We'll stick with the height example for now, but everything that follows is valid for any other example as well.

What we are going to to is take a list of numbers and encode it in a quantum state. We'll then manipulate the state in order to make fun things happen.

A quantum state is essentially described by a list of bit strings which each have a corresponding number. The bit strings are the possible outputs that you could get when measuring the state. The corresponding numbers are known as the amplitudes, and can be used to calculate the probabilities for the bit strings. Here we will ignore any other information encoded in these amplitudes, and just focus on the probabilities.

Our encoding of a list into a quantum state will then be as follows:
* We assign a bit string to each position in our list;
* Each number in our list is used as the probability for the bit string at that position.

This process has a few technicalities. The first is choosing how to assign bit strings to positions in the list. For this, we will want to respect the notion of neighbouring entries, as described above.

This will be most relevant when manipulating the state. We will mostly be using single qubit or controlled two qubit operations for these manipulations. The effect of these on simple bit strings is that they can only change a single bit value. For example, they can turn a `0000` into `0100` or `0001`, or turn `0110` into `1110`, and so on. Given this behaviour, we can think of bit strings as being neighbours if they differ on only one bit. And then we can make sure to assign neighbouring bit strings to neighbouring positions.

This is done by the following function. It creates a list of bit strings of a given length. The number of bits is determined by the length of the list, to ensure that bit string is used only once. The list is ordered such that neighbouring entries always differ on only a single bit.

In [ ]:
def make_line ( length ):
    # determine the number of bits required for at least `length` bit strings
    n = int(ceil(log(length)/log(2)))
    # start with the basic list of bit values
    line = ['0','1']
    # each application of the following process double the length of the list,
    # and of the bit strings it contains
    for j in range(n-1):
        # this is done in each step by first appending a reverse-ordered version of the current list
        line = line + line[::-1]
        # then adding a '0' onto the end of all bit strings in the first half
        for j in range(int(len(line)/2)):
            line[j] += '0'
        # and a '1' onto the end of all bit strings in the second half
        for j in range(int(len(line)/2),int(len(line))):
            line[j] += '1'
    return line

For example, here is a line long enough to encode the `height` list given as an example above.

In [66]:
line = make_line(6)
print(line)

['000', '100', '110', '010', '011', '111', '101', '001']


Note that here we requested a line of length 6, but a line of length 8 was given instead. This is because there are $2^n$ possible values for $n$-bit strings, and so the length of these lines will always be a power of 2. Since 2 bit strings would only cover lists of up to four entries, 3 bit strings are required to cover a list of 6 entries.

Now we have the bit strings, the next technicality is that of normalization. The trick of taking each number in the list to be a probability only works when those numbers are all non-negative, and when they all sum up to 1. To fix this we'll:
* Only use lists of non-negative numbers (okay, I admit, this is not much of a fix);
* Normalize the numbers to sum up to 1.

This is done in the following function, which takes a list of numbers and creates a corresponding quantum circuit.  In this circuit, the qubits are prepared in a state that encodes the list of numbers.

In [ ]:
def height_to_circuit( height ):
    
    line = make_line( len(height) )

    n = int(ceil(log(len(line))/log(2)))

    renorm = sqrt(sum(height))

    real_vec = [0]*(2**n)
    for j,h in enumerate(height):
        real_vec[int(line[j],2)] = sqrt(h)/renorm

    qc = QuantumCircuit(n)
    qc.initialize( real_vec )

    return qc

To get the list back out again, we use the following function.

However, note that this post-processing needs to somehow undo the normalization. However, the normalization will cause us to forget what the maximum value of the list was. We therefore simply assume that the maximum value was 1, and unnormalize accordingly.

If you didn't understand all this talk about normalization, the important fact is that the lists we get out will always have 1 as the maximum value.

In [40]:
def circuit_to_height( qc ):
    
    n = qc._n
    
    line = make_line( 2**n )

    real_vec = simulate(qc,get='statevector')
    
    height = [0]*(2**n)
    for j,amp in enumerate(real_vec):
        string = "{0:b}".format(j)
        string = '0'*(n-len(string)) + string
        k = line.index(string)
        height[k] = amp[0]**2
        
    max_prob = max(height)
    for j,h in enumerate(height):
        height[j] = h/max_prob
  
    return height

Now let's try it out. We already have a list of heights, we just need to encode them into a quantum circuit.

In [77]:
qc = height_to_circuit(height)

We can the pull them back out and see what happened.

In [78]:
circuit_to_height(qc)

[0.49999999999999994,
 0.6299999999999999,
 0.77,
 1.0,
 0.7499999999999998,
 0.49999999999999994,
 0.0,
 0.0]

Then come out pretty much as they went in, which is nice.

Now let's use some quantum operations to induce an interference effect, which will change the heights. For this we need to know how many qubits we have in our circuit, which is simply the length of the bit strings used.

In [79]:
n = len(line[0])

A simple effect is to apply a single qubit `ry` operation to each qubit by some angle `theta`. For `theta=0` we'd see no effect, whereas `theta=pi/2` would be quite drastic. We'll look at a relatively small but nevertheless non-trivial effect.

In [82]:
theta = pi/16

for j in range(n):
    qc.ry(theta,j)

To take a look at what it did to the heights, we just use the appropriate function.

In [81]:
circuit_to_height(qc)

[0.30929704993880464,
 0.6214947770424502,
 1.0,
 0.8328695336288192,
 0.8243127807658568,
 0.8325402300004812,
 6.355182238624218e-06,
 0.0006653107381321434]

And that's it. A very simple, quantum interference effect list manipulator thingy. Have fun!